In [149]:
import re
from pdfminer.high_level import extract_pages, extract_text

In [151]:
for page_layout in extract_pages("attestation.pdf"):
    for element in page_layout:
        print(extract_text(element))

TypeError: Unsupported input type: <class 'pdfminer.layout.LTTextBoxHorizontal'>

In [152]:
text =  extract_text("attestation.pdf")
print(text)

ATTESTATION D'HÉBERGEMENT À LA MAISEL SUDPARIS

Je soussigné, Philippe BEAULIEU, Directeur de la MAISEL SudParis, atteste que :
I, the undersigned Philippe BEAULIEU, MAISEL SudParis Manager, do here by certify that :

Nom : KAEWNOPARAT
Last name

Prénom : Kanawut
First name

Etudiant(e) à : Télécom SudParis
Student at

N°identifiant : 36344
Id number

pourra  bénéficier  d’un  logement  à  la  MAISEL  SudParis,  5,  rue  Charles  Fourier  91000  EVRY,  France
pendant  l’année  scolaire  2022-2023,  sous  réserve  qu’il  (elle)  réponde  à  nos  critères  d’attribution  et  qu’il
(elle) se conforme à notre procédure de demande d’hébergement.
will rent an accomodation at the  MAISEL SudParis, 5, rue Charles Fourier 91000 EVRY, France during
the  academic  year  2022-2023,  under  the  condition  that  he/she  complies  with  the  MAISEL  SudParis
admission rules.

Attestation établie à la demande de l’intéressé(e), pour servir et valoir ce que de droit.
Certificate established on behalf 

In [42]:
nom_pattern = re.compile("(?<=Nom :)(.*)")
prenom_pattern = re.compile("(?<=Prénom :)(.*)")
school_pattern = re.compile("(?<=Etudiant(e) à :)(.*)")
id_pattern =  re.compile("(?<=N°identifiant :)(.*)")

In [44]:
print(nom_pattern.search(text).group(1))
print(prenom_pattern.search(text).group(1))
print(id_pattern.search(text).group(1))

 KAEWNOPARAT
 Kanawut
 36344


In [46]:
print(text)

ATTESTATION D'HÉBERGEMENT À LA MAISEL SUDPARIS

Je soussigné, Philippe BEAULIEU, Directeur de la MAISEL SudParis, atteste que :
I, the undersigned Philippe BEAULIEU, MAISEL SudParis Manager, do here by certify that :

Nom : KAEWNOPARAT
Last name

Prénom : Kanawut
First name

Etudiant(e) à : Télécom SudParis
Student at

N°identifiant : 36344
Id number

pourra  bénéficier  d’un  logement  à  la  MAISEL  SudParis,  5,  rue  Charles  Fourier  91000  EVRY,  France
pendant  l’année  scolaire  2022-2023,  sous  réserve  qu’il  (elle)  réponde  à  nos  critères  d’attribution  et  qu’il
(elle) se conforme à notre procédure de demande d’hébergement.
will rent an accomodation at the  MAISEL SudParis, 5, rue Charles Fourier 91000 EVRY, France during
the  academic  year  2022-2023,  under  the  condition  that  he/she  complies  with  the  MAISEL  SudParis
admission rules.

Attestation établie à la demande de l’intéressé(e), pour servir et valoir ce que de droit.
Certificate established on behalf 

In [48]:
def get_info(pdf_file):
    text =  extract_text(pdf_file)
    lastname = nom_pattern.search(text).group(1)
    name = prenom_pattern.search(text).group(1)
    id = id_pattern.search(text).group(1)
    
    
    return {"lastname" : lastname,
            "name": name,
            "id":  id}

In [50]:
print(get_info("attestation.pdf"))

{'lastname': ' KAEWNOPARAT', 'name': ' Kanawut', 'id': ' 36344'}


## pdfquery

In [155]:
import pdfquery

In [156]:
pdf = pdfquery.PDFQuery('attestation.pdf')
pdf.load()

In [157]:
pdf.tree.write('pdfXML.txt', pretty_print = True)

In [117]:
33.431, 606.795, 126.964, 617.949

(33.431, 606.795, 126.964, 617.949)

In [159]:
def clean_text_data(text):
    return text.split(': ')[1]

In [160]:
s = pdf.pq('LTTextLineHorizontal:in_bbox("33.431, 606.795, 126.964, 617.949")').text()

In [218]:
lastname = pdf.pq('LTTextLineHorizontal:contains("Nom")').text()
name = pdf.pq('LTTextLineHorizontal:contains("Prénom")').text()
id  = pdf.pq('LTTextLineHorizontal:contains("id")').text()
school  = pdf.pq('LTTextLineHorizontal:contains("tudiant")').text()
address = "".join(pdf.pq('LTTextLineHorizontal:contains("pourra")').text().split(" ,")[1:])
year = pdf.pq('LTTextLineHorizontal:contains("l’année scolaire")').text()
verify_date = pdf.pq('LTTextLineHorizontal:in_bbox("33.431, 390.629, 138.602, 401.629")').text()[-10:]


In [197]:
year = pdf.pq('LTTextLineHorizontal:contains("l’année scolaire")').text()

In [219]:
pdf.pq('LTTextLineHorizontal:in_bbox("33.431, 390.629, 138.602, 401.629")').text()[-10:]

'04/07/2022'

In [209]:
def clean_year(text):
    output = re.compile("scolaire (.*),").search(text).group(1)
    return output

In [210]:
clean_year(year)

'2022-2023'

In [208]:
re.compile("scolaire (.*),").search(year).group(1)

'2022-2023'

In [201]:
" ".join(pdf.pq('LTTextLineHorizontal:contains("pourra")').text().split(", ")[1:])

'5 rue Charles Fourier 91000 EVRY France'

In [220]:
def use_pdfquery(pdf):
    pdf = pdfquery.PDFQuery('attestation.pdf')
    pdf.load()
    lastname = pdf.pq('LTTextLineHorizontal:contains("Nom")').text()
    name = pdf.pq('LTTextLineHorizontal:contains("Prénom")').text()
    id  = pdf.pq('LTTextLineHorizontal:contains("id")').text()
    school  = pdf.pq('LTTextLineHorizontal:contains("tudiant")').text()
    address  = " ".join(pdf.pq('LTTextLineHorizontal:contains("pourra")').text().split(", ")[1:])
    year = pdf.pq('LTTextLineHorizontal:contains("l’année scolaire")').text()
    certify_date = pdf.pq('LTTextLineHorizontal:in_bbox("33.431, 390.629, 138.602, 401.629")').text()[-10:]
    
    return {'lastname': clean_text_data(lastname),
            'name': clean_text_data(name),
            "id": clean_text_data(id),
            'school': clean_text_data(school),
            'address' : address,
            'academic_year': clean_year(year),
            'certify_date': certify_date}

In [221]:
use_pdfquery("attestation.pdf")

{'lastname': 'KAEWNOPARAT',
 'name': 'Kanawut',
 'id': '36344',
 'school': 'Télécom SudParis',
 'address': '5 rue Charles Fourier 91000 EVRY France',
 'academic_year': '2022-2023',
 'certify_date': '04/07/2022'}

## TSP Student Card

In [143]:
student_pdf = pdfquery.PDFQuery('attestation.pdf')
student_pdf.load()

In [144]:
student_pdf

In [145]:
student_pdf.tree.write('something.txt', pretty_print = True)

In [148]:
for page_layout in extract_pages("BDN_Kanawut_KAEWNOPARATsigned.pdf"):
    for element in page_layout:
        print(extract_text(element))

TypeError: Unsupported input type: <class 'pdfminer.layout.LTFigure'>

In [154]:
extract_text("TSP student card.pdf")

'\x0c'

In [222]:
mydict= {'lastname': 'KAEWNOPARAT', 'name': 'Kanawut', 'id': '36344', 'school': 'Télécom SudParis', 'address': '5 rue Charles Fourier 91000 EVRY France', 'academic_year': '2022-2023', 'certify_date': '04/07/2022'}

In [223]:
import pandas as pd

In [227]:
a = resutl = pd.DataFrame(mydict, index=[0])

In [228]:
a.to_clipboard(index=False)

In [231]:
a.to_clipboard()